# Generate Synthetic Video Titles

Code authored by: Shaw Talebi

### imports

In [1]:
import csv
import re
from itertools import combinations

from together import Together
from dotenv import load_dotenv
import os

In [2]:
# load vars from .env
load_dotenv()

# set together api key
client = Together(api_key=os.getenv("TOGETHER_API_KEY"))

### import ideas

In [3]:
# Open and read the CSV file
with open('data/ideas.csv', mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader) # skip first line

    # intialize list to store ideas
    idea_list = []
    
    for row in reader:
        idea_list.append(row[0])

In [4]:
len(idea_list)

114

### prompt template

In [5]:
template = lambda idea : f"""**YouTube Titles**:
- The 8 AI Skills That Will Separate Winners From Losers in 2025
- World's Lightest Solid!
- Why Are 96,000,000 Black Balls on This Reservoir?
- These 11 income streams made me $220,000 in 2024.
- I Make $15K/Month With 2 AI Apps
- Top 5 Reasons Not to Become a Data Analyst
- What Does a Data Analyst Actually Do?
- How I Would Become a Data Analyst if I had to Start Over in 2024 | 6 Month Plan
- How to learn to code FAST using ChatGPT (it's a game changer seriously)
- 6 Years of Studying Machine Learning in 26 Minutes
- My honest advice to someone who wants to be a data scientist
- Complete Python Pandas Data Science Tutorial! (Reading CSV/Excel files, Sorting, Filtering, Groupby)
- The Complete Machine Learning Roadmap
- My GPT-evaluator got 1000% better with this simple trick.
- The 5 paid subscriptions I actually use in 2025 as a Staff Software Engineer
- AI Explained at 5 Levels of Complexity
- Docker in 5 Minutes
- I asked 100 millionaires how to get rich–here's what happened.
- How I Build Projects (as an AI Engineer)
- AI Researcher critiques Claude 3.5 sonnet
- Data scientist explains how to predict the future
- I bought 10 data science courses so you don’t have to
- My AI Development Setup (From Scratch)
- How to Build a Resume Optimizer with AI (Code Walkthrough)
- I Quit My Job… Here’s How Much I Made 1 Year Later
- I Was Wrong About AI Consulting (what I learned)

--
Given the YouTube video idea write 5 engaging title ideas.

**Video Idea**: {idea}

**Additional Guidance**:
- Titles should be between 30 and 75 characters long
- Only return the title ideas, nothing else!
- Title ideas should be written as an ordered markdown list

"""

### generate titles

In [6]:
%%time
triplet_list = []
for idea in idea_list:
    # generate completion
    response = client.chat.completions.create(
        model="Qwen/Qwen2.5-7B-Instruct-Turbo",
        messages=[
            {"role": "user",
             "content": template(idea)
            },
    ],
        max_tokens=None,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["<|im_end|>"],
    )

    # parse completion (5 titles)
    response_raw = response.choices[0].message.content
    pattern = r"^\s*(?:[-*]|\d+\.)\s+(.+)$"
    title_list = re.findall(pattern, response_raw, re.MULTILINE)

    # generate all possible unique pairs
    title_pair_list = list(combinations(title_list, 2))
    
    # store all unique idea-title pairs in a list of dicts
    for a,b in title_pair_list:
        triplet_list.append({"idea":idea, "title_a": a, "title_b": b})

CPU times: user 359 ms, sys: 86 ms, total: 445 ms
Wall time: 1min 19s


In [7]:
len(triplet_list)

1140

### write list of dicts as .csv file

In [8]:
# Write to CSV
with open("data/idea-title_pairs.csv", mode="w", newline="", encoding="utf-8") as file:
    
    # Extract field names from the first dictionary
    fieldnames = triplet_list[0].keys()
    
    # Create DictWriter object
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    # Write the header row
    writer.writeheader()
    
    # Write data rows
    writer.writerows(triplet_list)
